In [1]:
import os
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# load_dotenv()
# os.getenv("OPENAI_API_KEY")

'sk-proj-yUkZgBfHGW6NR7SlbQVbT3BlbkFJFBNj1GjesfCmPmFbAlIX'

In [2]:
file_paths = ["yorum_soru_cevap.txt", "teknik_ozellikler.txt"]

docs = []

for file_path in file_paths:
    loader = TextLoader(file_path)
    docs.extend(loader.load())

In [3]:
docs

[Document(page_content='SORU-CEVAPLAR:\n\n\n\n\nMüşteri: iPhone 15 kaç GB hafızaya sahiptir?\nSatıcı: iPhone 15, 64 GB ve 128 GB hafıza seçenekleriyle mevcuttur efendim.\n\n\nMüşteri: Samsung Galaxy Note 10\'un ana kamerası kaç megapiksel?\nSatıcı: Samsung Galaxy Note 10, 50 MP\'lik ana kameraya sahiptir efendim.\n\n\nMüşteri: Bu ürün için kargo ücreti alıyor musunuz?\nSatıcı: Efendim, 200 TL ve üzeri alışverişlerinizde kargo ücretsizdir.\n\n\nMüşteri: Huawei P40 Pro\'nun ekran boyutu nedir?\nSatıcı: Huawei P40 Pro, 6.58 inç ekran boyutuna sahiptir efendim.\n\n\nMüşteri: Siparişimi kaç gün içinde teslim alabilirim?\nSatıcı: Siparişiniz, kargoya verildikten sonra 2-3 iş günü içinde teslim edilir efendim.\n\n\nMüşteri: Xiaomi Mi 11\'in batarya kapasitesi nedir?\nSatıcı: Xiaomi Mi 11, 4600 mAh batarya kapasitesine sahiptir efendim.\n\n\nMüşteri: Ürün iade süreci nasıl işliyor?\nSatıcı: Aldığınız ürünü, teslim aldıktan sonra 14 gün içinde iade edebilirsiniz efendim. İade süreci için müşter

In [4]:
#split document into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512, chunk_overlap=100, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

In [5]:
all_splits

[Document(page_content="SORU-CEVAPLAR:\n\n\n\n\nMüşteri: iPhone 15 kaç GB hafızaya sahiptir?\nSatıcı: iPhone 15, 64 GB ve 128 GB hafıza seçenekleriyle mevcuttur efendim.\n\n\nMüşteri: Samsung Galaxy Note 10'un ana kamerası kaç megapiksel?\nSatıcı: Samsung Galaxy Note 10, 50 MP'lik ana kameraya sahiptir efendim.\n\n\nMüşteri: Bu ürün için kargo ücreti alıyor musunuz?\nSatıcı: Efendim, 200 TL ve üzeri alışverişlerinizde kargo ücretsizdir.", metadata={'source': 'yorum_soru_cevap.txt', 'start_index': 0}),
 Document(page_content="Müşteri: Huawei P40 Pro'nun ekran boyutu nedir?\nSatıcı: Huawei P40 Pro, 6.58 inç ekran boyutuna sahiptir efendim.\n\n\nMüşteri: Siparişimi kaç gün içinde teslim alabilirim?\nSatıcı: Siparişiniz, kargoya verildikten sonra 2-3 iş günü içinde teslim edilir efendim.\n\n\nMüşteri: Xiaomi Mi 11'in batarya kapasitesi nedir?\nSatıcı: Xiaomi Mi 11, 4600 mAh batarya kapasitesine sahiptir efendim.", metadata={'source': 'yorum_soru_cevap.txt', 'start_index': 405}),
 Document(

In [6]:
len(all_splits)

174

In [7]:
# store the data into a vector store (we will be using FAISS for this example)
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(all_splits, embeddings)

In [8]:
query = "Apple Watch Ultra 2'nin batarya süresi ne kadar?"
response_db = db.similarity_search(query)

In [9]:
response_db

[Document(page_content='"Uzun yürüyüş ve doğa turlarımda Apple Watch Ultra 2\'nin 542 mAh bataryası, 60 saatlik bekleme süresi ile beni etkiledi. GPS ve altimetre gibi özellikleri aktif kullanmama rağmen, bataryanın dayanıklılığı mükemmel. Sadece 30 dakikada %50 şarj edebilmek, doğa turlarımda vazgeçilmez hale getiriyor."', metadata={'source': 'yorum_soru_cevap.txt', 'start_index': 21597}),
 Document(page_content='"Apple Watch Ultra 2’nin 542 mAh bataryası, tüm gün boyunca yoğun kullanımda bile dayanıyor. Hızlı şarj özelliği ile saati sadece 30 dakikada %50 oranında şarj edebiliyorum, bu da iş seyahatlerimde büyük bir avantaj sağlıyor."\n\n"Ultra 2’nin LTE özelliği sayesinde telefonuma ihtiyaç duymadan arama yapabiliyor ve mesaj gönderebiliyorum. Ayrıca, 32GB dahili depolama alanı ile müzik ve uygulamalar için yeterli alan sağlıyor."', metadata={'source': 'yorum_soru_cevap.txt', 'start_index': 17790}),
 Document(page_content='"Apple Watch Ultra 2’nin batarya ömrü yoğun kullanımda bile 

In [10]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [11]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125",
                 temperature=1,
                 )

In [12]:
template = """Sen teknolojik ürünler satan bir e-ticaret sitesinde çalışan ve ürünlerle ilgili soruları cevaplayan bir asistansın.
Sondaki soruyu yanıtlamak için sadece aşağıdaki bağlamı kullanın.
Eğer cevabı bilmiyorsanız, sadece bilmediğinizi söyleyin, bir cevap uydurmaya çalışmayın.

Bağlam: {context}

Soru: {question}

Yardımsever Asistan:"""

custom_rag_prompt = PromptTemplate.from_template(template)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [13]:
query = "Kredi kartına taksit imkanı var mı?"


rag_response = rag_chain.invoke(query)
print(rag_response)

Evet efendim, kredi kartına 12 aya kadar taksit imkanı sunuyoruz.


In [14]:
### Contextualize question ###
contextualize_q_system_prompt = (
"""Sohbet geçmişi ve en son kullanıcı sorusunu göz önünde bulundurularak sohbet geçmişine atıfta bulunabilecek bir soru verildiğinde
bu soruyu, sohbet geçmişi olmadan da anlaşılabilecek bağımsız bir soru olarak formüle edin. Soruyu CEVAPLAMAYIN.
Sadece gerektiğinde yeniden formüle edin ve aksi takdirde olduğu gibi geri döndürün."""
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [15]:
### Answer question ###
system_prompt = ("""Sen teknolojik ürünler satan bir e-ticaret sitesinde çalışan ve ürünlerle ilgili soruları cevaplayan bir asistansın.
Soruyu yanıtlamak için sadece aşağıdaki bağlamı kullanın.
Eğer cevabı bilmiyorsanız, sadece bilmediğinizi söyleyin, bir cevap uydurmaya çalışmayın.

Context: {context}
"""
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [16]:
### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [17]:
question_1 = "Lenovo Yoga 520'nin ekranı kaç inch?"

conversational_rag_chain.invoke(
    {"input": question_1},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

"Lenovo Yoga 520'nin ekranı 14 inch büyüklüğündedir."

In [18]:
question_2 = "Peki bu bilgisayarın işlemcisi nedir?"

conversational_rag_chain.invoke(
    {"input": question_2},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

"Lenovo Yoga 520'nin işlemcisi Intel Core i5-7200U'dur."

In [19]:
import gradio as gr

def interact(query, history):
    return conversational_rag_chain.invoke({"input": query},config={"configurable": {"session_id": "abc123"}},)["answer"]

demo = gr.ChatInterface(
    fn=interact,
    examples=[
        "Lenovo Yoga 520'nin ekranı kaç inch?",
        "Kredi kartına taksit imkanı var mı?",
        "Kardeşim bir koşucu, ona ne hediye alabilirim?",
        "Apple Watch Ultra 2'nin sarjı kaç saat gidiyor?",
        "Çocuğuma hediye olarak Apple Watch Ultra 2 alabilir miyim?"
    ],
    title="E-Ticaret Soru-Cevap Uygulaması",
    description="Bu uygulama ile dilediğiniz ürün hakkındaki soruları Yapay Zeka modelimize sorup cevap alabilirsiniz."
)

demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://d7f7175fc4cd5e548f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
